In [1]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')

# Launch DPS for tile_atl08.py

In [2]:
import os
import geopandas
import pandas as pd
import json

/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
def get_stack_fn(stack_list_fn, in_tile_num):
    # Find most recent topo/Landsat stack path for tile in list of stack paths from *tindex_master.csv
    all_stacks_df = pd.read_csv(stack_list_fn)
    stack_for_tile = all_stacks_df[all_stacks_df['location'].str.contains("_"+str(in_tile_num))]
    [print(i) for i in stack_for_tile.path.to_list()]
    stack_for_tile_fn = stack_for_tile.path.to_list()[0]
    if len(stack_for_tile)==0:
        stack_for_tile_fn = None
    return(stack_for_tile_fn)

# nmt added: code that returns df of landsat locations and tile number
# This is basically CountOutput.py
def get_stack_df(dps_dir, TYPE, dps_year):
    
    if "Landsat" in TYPE:
        root = f"/projects/my-private-bucket/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/{dps_year}/"
        ends_with_str = "_dps.tif"
    if "Topo" in TYPE:
        root = f"/projects/my-private-bucket/dps_output/do_topo_stack_3-1-5_ubuntu/ops/{dps_year}/"
        ends_with_str = "_stack.tif"
    if "ATL08" in TYPE:
        root = f"/projects/my-private-bucket/dps_output/run_extract_ubuntu/ops/{dps_year}/"
        ends_with_str = "0m.csv"
            
    df = pd.DataFrame(columns=['location', 'tile_num'])

    for dir, subdir, files in os.walk(root):
        for fname in files:
            if fname.endswith(ends_with_str): 
                 
                tile_num = fname.split('_')[1]
                   
                if "ATL08" in TYPE:
                    df = df.append({'location':os.path.join(dir+"/", fname)},ignore_index=True)
                else:
                    df = df.append({'location':os.path.join(dir+"/", fname), 'tile_num':tile_num},ignore_index=True)
        
    return df

## Get df's from tindex_master csvs for Topo and Landsat tiles

These are already done after 'build_tindex_master.py'

In [4]:
import os
os.system("python /projects/icesat2_boreal/lib/build_tindex_master.py --type Topo")
os.system("python /projects/icesat2_boreal/lib/build_tindex_master.py --type Landsat")

0

In [5]:
# Topo and Landsat tindex_master csvs from CountOutput.py
topo_tindex = "/projects/shared-buckets/nathanmthomas/DPS_tile_lists/Topo_tindex_master.csv"
landsat_tindex = "/projects/shared-buckets/nathanmthomas/DPS_tile_lists/Landsat_tindex_master.csv"

if True:
    if os.path.isfile(landsat_tindex) and os.path.isfile(topo_tindex):
        ls8_df = pd.read_csv(landsat_tindex)
        topo_df = pd.read_csv(topo_tindex)
    else:
        s3_stem = 'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/nathanmthomas'
        local_stem = '/projects/my-private-bucket'

        ls8_root =  s3_stem + '/dps_output/do_landsat_stack_3-1-2_ubuntu'
        topo_root = s3_stem + '/dps_output/do_topo_stack_3-1-5_ubuntu'

        ls8_df = get_stack_df(ls8_root, "Landsat", 2020)
        topo_df = get_stack_df(topo_root, "Topo", 2020)

## Get tile nums for coincident Topo and Landsat tiles

In [6]:
# Model-read subset of tiles for which Topo and Landsat coincide
model_ready_tiles_topo = "/projects/shared-buckets/nathanmthomas/DPS_tile_lists/model_ready_tiles_topo_paths.csv"
model_ready_tiles_landsat = "/projects/shared-buckets/nathanmthomas/DPS_tile_lists/model_ready_tiles_landsat_paths.csv"

if os.path.isfile(model_ready_tiles_topo) or os.path.isfile(model_ready_tiles_landsat):
    topo_sub_df = pd.read_csv(model_ready_tiles_topo)
    ls8_sub_df = pd.read_csv(model_ready_tiles_landsat)
else:
    # added by nmt: get filenames of co-incident landsat and topo
    topo_sub_df = pd.DataFrame(columns=['local_path','tile_num'])
    ls8_sub_df = pd.DataFrame(columns=['local_path','tile_num'])

    for i in range(len(ls8_df['tile_num'])):
        ls_tile_num = ls8_df['tile_num'][i]
        for j in range(len(topo_df['tile_num'])):
            topo_tile_num = topo_df['tile_num'][j]
            if ls_tile_num == topo_tile_num:
                # Only need to choose one, but we'll do 2 and then check
                ls8_sub_df = ls8_sub_df.append({'local_path':ls8_df['local_path'][i],'tile_num':ls8_df['tile_num'][i].astype(int)}, ignore_index=True)
                topo_sub_df = topo_sub_df.append({'local_path':topo_df['local_path'][j],'tile_num':topo_df['tile_num'][j].astype(int)}, ignore_index=True)

    #ls8_sub_df['tile_num'] = ls8_sub_df['tile_num'].astype(float, errors = 'raise')

    topo_sub_df.to_csv( model_ready_tiles_topo, index=False, encoding='utf-8-sig')
    ls8_sub_df.to_csv( model_ready_tiles_landsat, index=False, encoding='utf-8-sig')
    
print(ls8_sub_df.head())
print(topo_sub_df.head())
print(len(ls8_sub_df),len(topo_sub_df))

                                          local_path  tile_num
0  /projects/my-private-bucket/dps_output/do_land...       986
1  /projects/my-private-bucket/dps_output/do_land...       987
2  /projects/my-private-bucket/dps_output/do_land...       979
3  /projects/my-private-bucket/dps_output/do_land...       984
4  /projects/my-private-bucket/dps_output/do_land...       982
                                          local_path  tile_num
0  /projects/my-private-bucket/dps_output/do_topo...       986
1  /projects/my-private-bucket/dps_output/do_topo...       987
2  /projects/my-private-bucket/dps_output/do_topo...       979
3  /projects/my-private-bucket/dps_output/do_topo...       984
4  /projects/my-private-bucket/dps_output/do_topo...       982
4465 4465


In [7]:
topo_sub_df = pd.read_csv("/projects/shared-buckets/nathanmthomas/DPS_tile_lists/model_ready_tiles_topo_paths.csv")
INPUT_TILE_NUM_LIST = topo_sub_df['tile_num'].values.astype(int).tolist()
len(INPUT_TILE_NUM_LIST)

4465

## Run a DPS job

In [8]:
#"https://maap-ops-workspace.s3.amazonaws.com.com/lduncanson"
#s3_stem = 'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/nathanmthomas'
#local_stem = '/projects/my-private-bucket'
RUN_DPS  = False
if RUN_DPS:
    ##################################
    #Test DPS submission on a single file
    for i, INPUT_TILE_NUM in enumerate(INPUT_TILE_NUM_LIST):
        DPS_num = i+1
        if True:
            in_param_dict = {
                                'in_tile_num': INPUT_TILE_NUM,
                                'in_tile_fn': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg',
                                'in_tile_layer': 'grid_boreal_albers90k_gpkg',
                                'csv_list_fn': 's3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_tindex_master.csv',
                                'topo_stack_list_fn': 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Topo_tindex_master.csv',
                                'landsat_stack_list_fn': 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Landsat_tindex_master.csv',
                                'user_stacks': 'nathanmthomas',
                                'user_atl08': 'lduncanson'
            }

            submit_result = maap.submitJob(
                identifier='run_tile_atl08',
                algo_id='run_tile_atl08_ubuntu',
                version='master',
                username='lduncanson', # username needs to be the same as whoever created the workspace
                queue='maap-dps-worker-8gb',
                **in_param_dict
            )

        #submit_result = 'submit test'
        if DPS_num in [1, 100,500,1000,3000, len(INPUT_TILE_NUM_LIST)]:
           print(f"DPS run num: {DPS_num}, tile num: {INPUT_TILE_NUM}, job info: {submit_result}") 
 

Now build the tindex csv to show paths of all output

In [ ]:
import os
os.system("python /projects/icesat2_boreal/lib/build_tindex_master.py --type ATL08_filt")

# Review the DPS outputs

In [ ]:
def local_to_s3(url, user='nathanmthomas'):
    ''' A Function to convert local paths to s3 urls'''
    return url.replace('/projects/my-private-bucket', f's3://maap-ops-workspace/{user}')
def local_to_s3(url, user = 'nathanmthomas', type='public'):
    ''' A Function to convert local paths to s3 urls'''
    if type is 'public':
        return url.replace(f'/projects/my-{type}-bucket', f's3://maap-ops-workspace/shared/{user}')
    else:
        

In [ ]:
tindex_master_fn = 's3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv'

# Build up a dataframe from the list of dps output files
tindex_master = pd.read_csv(tindex_master_fn)
tindex_master['s3'] = [local_to_s3(local_path, user='lduncanson') for local_path in tindex_master['local_path']]
print(tindex_master.head())

# Get boreal tiles
boreal_tile_index_path = '/projects/shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg'
boreal_tile_index = geopandas.read_file(boreal_tile_index_path)
boreal_tile_index.astype({'layer':'int'})
boreal_tile_index.rename(columns={"layer":"tile_num"}, inplace=True)
boreal_tile_index["tile_num"] = boreal_tile_index["tile_num"].astype(int)

bad_tiles = [3540,3634,3728,3823,3916,4004] #Dropping the tiles near antimeridian that reproject poorly.

# For some reason, doing this causes 'MosaicJSON.from_features()' to fail...(below)
if True:
    # Remove bad tiles
    boreal_tile_index = boreal_tile_index[~boreal_tile_index['tile_num'].isin(bad_tiles)]
    
print(boreal_tile_index.head())

# Identify duplicate tiles

In [ ]:
import collections
duplicate_tiles = [item for item, count in collections.Counter(tindex_master["tile_num"]).items() if count > 1]
print(duplicate_tiles)


# Identify matching tiles

In [ ]:
# Select the rows we have results for
tile_matches = boreal_tile_index.merge(tindex_master[~tindex_master['tile_num'].isin(bad_tiles)], how='right', on='tile_num')
print(tile_matches.shape)

tile_matches_duplicates = boreal_tile_index.merge(tindex_master[tindex_master['tile_num'].isin(duplicate_tiles)], how='right', on='tile_num')
print(tile_matches_duplicates.shape)

tile_matches_geojson_string = tile_matches.to_crs("EPSG:4326").to_json()
tile_matches_geojson = json.loads(tile_matches_geojson_string)

tile_matches_duplicates_geojson_string = tile_matches_duplicates.to_crs("EPSG:4326").to_json()
tile_matches_duplicates_geojson = json.loads(tile_matches_duplicates_geojson_string)

## Build a MosaicJSON

In [ ]:
from typing import Dict

from cogeo_mosaic.mosaic import MosaicJSON
from cogeo_mosaic.backends import MosaicBackend

def get_accessor(feature: Dict):
    """Return specific feature identifier."""
    return feature["properties"]["s3"]

In [ ]:
mosaic_json = 's3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master_mosaic.json' 

mosaicdata = MosaicJSON.from_features(tile_matches_geojson.get('features'), minzoom=6, maxzoom=18, accessor=get_accessor)

with MosaicBackend(mosaic_json, mosaic_def=mosaicdata) as mosaic:
    mosaic.write(overwrite=True)
    
# URL to give tiler is
# s3://maap-ops-workspace/shared/nathanmthomas/Landsat/landsat8_mosaic.json

## View the Results with Folium

In [ ]:
from folium import Map, TileLayer, GeoJson, LayerControl, Icon, Marker

In [ ]:
# Setup the mosaic tiling
tiler_base = "https://jqsd6bqdsf.execute-api.us-west-2.amazonaws.com/"
tiler_mosaic =  "".join([tiler_base, "mosaicjson/tiles/{z}/{x}/{y}"])

# Get a basemap
tiler_basemap = "http://services.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}"

# Get Vector layers
boreal_geojson = '/projects/shared-buckets/nathanmthomas/boreal.geojson' 
boreal = geopandas.read_file(boreal_geojson)
ecoboreal_geojson = '/projects/shared-buckets/nathanmthomas/Ecoregions2017_boreal_m.geojson'
ecoboreal = geopandas.read_file(ecoboreal_geojson)

# Reproject Vector Layers
p1, p2, clat, clon = [50, 70, 40, 160]
proj_str_aea = '+proj=aea +lat_1={:.2f} +lat_2={:.2f} +lat_0={:.2f} +lon_0={:.2f}'.format(p1, p2, clat, clon)
ecoboreal_aea = ecoboreal.to_crs(proj_str_aea)
# Apply a buffer
ecoboreal_aea_buf = ecoboreal_aea["geometry"].buffer(1e5)
# Go back to GCS
ecoboreal_buf = ecoboreal_aea_buf.to_crs(boreal_tile_index.crs)

# Style Vector Layers
ecoboreal_style = {'fillColor': 'orange', 'color': 'orange'}
boreal_style = {'fillColor': 'green', 'color': 'green'}
boreal_subset_style = {'fillColor': 'red', 'color': 'red'}

# Map the Layers
#------------------
m1 = Map(
    tiles="Stamen Terrain"
)

#GeoJson(ecoboreal_aea_buf, name="Boreal extent from Ecoregions", style_function=lambda x:ecoboreal_style).add_to(m1)
GeoJson(boreal, name="Boreal extent", style_function=lambda x:boreal_style).add_to(m1)

bbox_style = {'fillColor': '#ff7f00', 'color': '#ff7f00'}
all_tiles = GeoJson(
    data=boreal_tile_index.to_crs("EPSG:4326").to_json(),
    style_function=lambda x:bbox_style,
    name="Boreal tiles"
).add_to(m1)

atl08_style = {'fillColor': '#377eb8', 'color': '#377eb8'}
atl08_dup_style = {'fillColor': 'red', 'color': 'red'}

atl08_geojson = GeoJson(
        data=tile_matches_geojson,
        style_function=lambda x:atl08_style,
        name="ATL08-filt DPS subset of Boreal tiles"
    ).add_to(m1)

atl08_duplicates_geojson = GeoJson(
        data=tile_matches_duplicates_geojson,
        style_function=lambda x:atl08_dup_style,
        name="Duplicates: ATL08-filt DPS subset of Boreal tiles"
    ).add_to(m1)
if True:
    
    basemap = TileLayer(
        tiles=tiler_basemap,
        opacity=1,
        name="World gray basemap",
        attr="MAAP",
        overlay=True
    )
    landsat_tiles = f"{tiler_mosaic}?url=s3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Landsat_mosaic.json&rescale=0.01,0.5&bidx=6&colormap_name=viridis"
    landsat_tiles_layer = TileLayer(
        tiles=landsat_tiles,
        opacity=1,
        name="landsat covars",
        attr="MAAP",
        overlay=True
    )
    topo_tiles = f"{tiler_mosaic}?url=s3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Topo_mosaic.json&rescale=0,1&bidx=3&colormap_name=bone"
    topo_tiles_layer = TileLayer(
        tiles=topo_tiles,
        opacity=1,
        name="topo covars",
        attr="MAAP",
        overlay=True
    )
    basemap.add_to(m1)
    landsat_tiles_layer.add_to(m1)
    topo_tiles_layer.add_to(m1)

LayerControl().add_to(m1)

m1